# XGBoost Regressor

## Mục tiêu
  - Xây dựng mô hình XGBoost Regressor để dự đoán tuổi thọ trung bình.

  - Sử dụng dữ liệu đã được tiền xử lý từ `data/processed/`.

  - Tối ưu hóa siêu tham số bằng 5-Fold Cross-Validation.

  - Do MAE đo sai số theo đúng đơn vị thực tế (năm tuổi thọ). Ta sử dụng tiêu chuẩn là độ đo MAE để chọn siêu tham số cuối cùng.


## Giới thiệu
XGBoost (Extreme Gradient Boosting) là một thuật toán boosting mạnh mẽ, nổi tiếng với:
- Hiệu suất cao.

- Khả năng tổng quát tốt.

- Tốc độ huấn luyện nhanh.

- Khả năng regularization mạnh để chống overfitting.

XGBoost thường là lựa chọn hàng đầu trong nhiều cuộc thi Kaggle và các bài toán thực tế về dữ liệu dạng bảng (tabular data).

### **Ưu điểm:**
- Hiệu năng rất cao: thường vượt trội so với Random Forest và đôi khi tốt hơn cả LightGBM khi dữ liệu không quá lớn.

- Regularization mạnh (L1 & L2) giúp kiểm soát overfitting tốt.

- Linh hoạt: hỗ trợ nhiều loại hàm mất mát, phù hợp cho cả bài toán regression lẫn classification.

- Xử lý missing values thông minh: XGBoost có cơ chế tự học cách rẽ nhánh tối ưu cho giá trị khuyết.

### **Nhược điểm:**
- Chậm hơn LightGBM trên dữ liệu lớn.

- Dễ overfitting nếu cây có độ sâu lớn hoặc learning rate cao.

- Không xử lý trực tiếp dữ liệu categorical. Do đó cần phải mã hóa bằng one-hot, label encoding,...

# Bước 1 - Import các thư viện cần thiết

### 1.1. Cài đặt thư viện xgboost (nếu cần)

In [1]:
!pip install xgboost --quiet

### 1.2. Import các thư viện cơ bản khác

In [1]:
import os
import joblib
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from scipy.stats import randint, uniform, loguniform
RANDOM_STATE = 42

## Bước 2 - Đọc dữ liệu đã tiền xử lý

Đọc dữ liệu từ `../data/processed/train.csv` và hiển thị thông tin cơ bản.

In [2]:
train_df = pd.read_csv('../data/processed/train.csv')

print("THÔNG TIN DỮ LIỆU")
print("="*60)
print(f"Kích thước tập train: {train_df.shape}")
print("\n5 dòng đầu của tập train:")
train_df.head()

THÔNG TIN DỮ LIỆU
Kích thước tập train: (3124, 13)

5 dòng đầu của tập train:


,country_name,country_code,year,population,pop_growth,life_expectancy,gdp_per_capita,gdp_growth,sanitation,electricity,water_access,co2_emissions,labor_force
0,Denmark,DNK,2017,-0.203264,-0.418953,81.102439,1.682694,-0.047741,1.673510,0.642797,0.754689,0.118599,0.069223
1,"Korea, Dem. People's Rep.",PRK,2017,-0.056076,-0.530921,73.034000,-0.412271,0.028651,-0.245951,-1.294730,0.439985,-0.216848,2.087276
2,Madagascar,MDG,2008,-0.091998,1.007790,61.992000,-0.607170,0.547296,-1.651029,-2.194047,-2.540432,-0.528445,2.542268
3,Greece,GRC,2018,-0.166799,-0.945727,81.787805,0.170491,-0.209157,1.356075,0.642797,0.754689,0.161554,-1.033944
4,South Sudan,SSD,2019,-0.169071,1.000983,58.129000,-0.412271,0.028651,-1.375450,-2.593358,-2.691194,-0.278341,1.279234


## Bước 3 - Chuẩn bị dữ liệu cho mô hình

Tách biến mục tiêu (`life_expectancy`) khỏi các đặc trưng. Loại bỏ các cột không cần thiết như `country_name`, `country_code`

In [3]:
# Định nghĩa các cột dùng để dự đoán
feature_cols = [col for col in train_df.columns 
                if col not in ['life_expectancy', 'country_name', 'country_code']]

# Tách X và y cho từng tập
X_train = train_df[feature_cols]
y_train = train_df['life_expectancy']

print("THÔNG TIN CÁC TẬP DỮ LIỆU")
print("="*60)
print(f"Số lượng đặc trưng: {len(feature_cols)}")
print(f"\nCác đặc trưng được sử dụng:")
for i, col in enumerate(feature_cols, 1):
    print(f"  {i}. {col}")

print(f"\nKích thước X_train: {X_train.shape}")
print(f"Kích thước y_train: {y_train.shape}")

THÔNG TIN CÁC TẬP DỮ LIỆU
Số lượng đặc trưng: 10

Các đặc trưng được sử dụng:
  1. year
  2. population
  3. pop_growth
  4. gdp_per_capita
  5. gdp_growth
  6. sanitation
  7. electricity
  8. water_access
  9. co2_emissions
  10. labor_force

Kích thước X_train: (3124, 10)
Kích thước y_train: (3124,)


## Bước 4 - Xây dựng và huấn luyện mô hình XGBoost

### 4.1. Sử dụng RandomizedSearchCV với 5-Fold Cross-Validation để tìm kiếm siêu tham số tối ưu cho mô hình XGBoost.

In [ ]:
# Định nghĩa không gian siêu tham số để tìm kiếm
param_distributions_xgb = {
    'n_estimators': randint(200, 2000),
    'max_depth': randint(3, 12),
    'learning_rate': loguniform(1e-3, 3e-1),
    'subsample': uniform(0.6, 0.4),
    'colsample_bytree': uniform(0.5, 0.5),
    'gamma': loguniform(1e-3, 10),
    'reg_alpha': loguniform(1e-3, 10),
    'reg_lambda': loguniform(1e-3, 10),
}

xgb = XGBRegressor(objective='reg:squarederror', random_state=RANDOM_STATE, verbosity=0)

# Tạo RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=param_distributions_xgb,
    n_iter=60,
    scoring='neg_mean_absolute_error',
    cv=5,
    random_state=RANDOM_STATE,
    verbose=2
)

# Huấn luyện tìm kiếm siêu tham số
random_search.fit(X_train, y_train)

Fitting 5 folds for each of 60 candidates, totalling 300 fits
[CV] END colsample_bytree=0.6872700594236812, gamma=6.351221010640703, learning_rate=0.06504856968981275, max_depth=7, n_estimators=1838, reg_alpha=0.06071989493441298, reg_lambda=0.002511306167739001, subsample=0.7836995567863468; total time=   0.4s
[CV] END colsample_bytree=0.6872700594236812, gamma=6.351221010640703, learning_rate=0.06504856968981275, max_depth=7, n_estimators=1838, reg_alpha=0.06071989493441298, reg_lambda=0.002511306167739001, subsample=0.7836995567863468; total time=   0.5s
[CV] END colsample_bytree=0.6872700594236812, gamma=6.351221010640703, learning_rate=0.06504856968981275, max_depth=7, n_estimators=1838, reg_alpha=0.06071989493441298, reg_lambda=0.002511306167739001, subsample=0.7836995567863468; total time=   0.5s
[CV] END colsample_bytree=0.6872700594236812, gamma=6.351221010640703, learning_rate=0.06504856968981275, max_depth=7, n_estimators=1838, reg_alpha=0.06071989493441298, reg_lambda=0.002

### 4.2. Hiển thị siêu tham số tối ưu và đánh giá hiệu suất của mô hình tốt nhất.

In [ ]:
best_xgb = random_search.best_estimator_
best_params = random_search.best_params_

print("="*60)
print("SIÊU THAM SỐ TỐI ƯU")
print("="*60)
for param, value in best_params.items():
    print(f"  {param:20s}: {value}")

SIÊU THAM SỐ TỐI ƯU
  colsample_bytree    : 0.8212096391031578
  gamma               : 0.0012765717873951684
  learning_rate       : 0.028251215820188597
  max_depth           : 6
  n_estimators        : 1887
  reg_alpha           : 0.5112971471524528
  reg_lambda          : 0.19654779079114093
  subsample           : 0.651400141293564


### 4.3. In ra CV MAE loss

In [ ]:
y_train_pred = best_xgb.predict(X_train)

train_mae = mean_absolute_error(y_train, y_train_pred)
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
train_r2 = r2_score(y_train, y_train_pred)

print("KẾT QUẢ MÔ HÌNH TỐI ƯU")
print("="*60)
print("Model: XGBoost")
print(f"RMSE loss: {train_rmse:.4f}")
print(f"MAE loss : {train_mae:.4f}")
print(f"R2 score : {train_r2:.4f}")

KẾT QUẢ MÔ HÌNH TỐI ƯU
Model: XGBoost
RMSE loss: 0.1298
MAE loss : 0.0971
R2 score : 0.9998


### 4.4. Lưu mô hình đã huấn luyện

Lưu mô hình XGBoost đã tối ưu vào file để sử dụng sau này.

In [ ]:
os.makedirs('../models/7_xgboost', exist_ok=True)
model_path = '../models/7_xgboost/xgboost.pkl.gz'
joblib.dump(best_xgb, model_path, compress=('gzip', 9))
print(f"Saved best XGBoost model to: {model_path}")

Saved best XGBoost model to: ../models/7_xgboost/xgboost.pkl.gz


## Kết luận

### Tổng kết:
- Mô hình XGBoost Regressor đã được xây dựng và tối ưu hóa thành công.

- RandomizedSearchCV với 5-Fold CV đã tìm được bộ siêu tham số tối ưu
